In [1]:
import numpy as np
from copy import copy
import tensorflow as tf
from importlib import reload
from random import randint

import environment as env
import agent
import agents.random_agent as ra
import agents.rl_3_no_hidden as rl

In [60]:
reload(env)
reload(agent)
reload(ra)
reload(rl)

<module 'agents.rl_3_no_hidden' from 'C:\\Users\\abeol\\Git\\Deep-Notakto\\deepnotakto\\agents\\rl_3_no_hidden.py'>

In [51]:
e = env.Env(3)
a1 = rl.RL3NoHidden()
a2 = ra.RandomAgent()
e.play_cvc(a1, a2, True)

Turn #0

X O O 
O O O 
O O O 

Turn #1

X O O 
O O X 
O O O 

Turn #2

X O O 
O O X 
O O X 

Turn #3

X X O 
O O X 
O O X 

Turn #4

X X O 
O O X 
O O X 

Player attempted illegal move
Turn #5

X X O 
O O X 
O X X 

Turn #6

X X O 
O O X 
O X X 

Player attempted illegal move
Turn #7

X X O 
O X X 
O X X 

Player 1 Wins!


In [61]:
env.Env(4).play_hvh()


O O O O 
O O O O 
O O O O 
O O O O 

Player 1
Next Piece: 1 1

X O O O 
O O O O 
O O O O 
O O O O 

Player 2
Next Piece: 2 2

X O O O 
O X O O 
O O O O 
O O O O 

Player 1
Next Piece: 3 3

X O O O 
O X O O 
O O X O 
O O O O 

Player 2
Next Piece: 4 4

X O O O 
O X O O 
O O X O 
O O O X 

GAME OVER! Player 1 Wins!
